In [1]:
import os
import sys

import timm
import torchvision.transforms as T
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import hydra
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from dotenv import load_dotenv, dotenv_values

sys.argv = ['']
# 환경변수 읽기
if (python_path := dotenv_values().get('PYTHONPATH')) and python_path not in sys.path: sys.path.append(python_path)

from src.models.DNN import SimpleDNN
from src.utils import config

# 데이터 준비 함수
def prepare_data(data_dir=config.DATA_DIR, batch_size=64, num_workers=4):
    # 데이터 전처리
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.5], std=[0.5]) 
    ])
    
    # 훈련 데이터셋
    train_dataset = MNIST(data_dir, train=True, download=True, transform=transform)
    train_dataset, val_dataset = random_split(train_dataset, [55000, 5000])
    
    # 테스트 데이터셋
    test_dataset = MNIST(data_dir, train=False, download=True, transform=transform)
    
    # DataLoader 생성
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=num_workers
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=num_workers
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=num_workers
    )

    return train_loader, val_loader, test_loader

@hydra.main(config_path=config.CONFIGS_DIR, config_name="config", version_base=None)
def main(cfg):
    print(cfg)

    pl.seed_everything(42)
    
    # 데이터 로더 준비
    train_loader, val_loader, test_loader = prepare_data(
        data_dir=config.DATA_DIR,
        batch_size=cfg.data.batch_size,
        num_workers=4
    )

    model = SimpleDNN(cfg)

    trainer = Trainer(default_root_dir=config.OUTPUTS_DIR, max_epochs=cfg.trainer.max_epochs, accelerator=cfg.trainer.accelerator)
    
    # 훈련
    trainer.fit(model, train_loader, val_loader)
    
    # 테스트
    test_results = trainer.test(model, test_loader)

    # 테스트 결과 출력
    if test_results and len(test_results) > 0:
        test_acc = test_results[0].get('test_acc', 0.0)
        print(f"최종 테스트 정확도: {test_acc:.4f}")
    else:
        print("테스트 결과를 가져올 수 없습니다.")

if __name__ == "__main__":
    main()

/data/ephemeral/home/mywork/python_work/git/gx-train/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
Seed set to 42
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/data/ephemeral/home/mywork/python_work/git/gx-train/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For th

{'data': {'data_dir': './CIFAR10_DATASET', 'batch_size': 64, 'valid_split': 0.2}, 'model': {'model_name': 'dnn', 'hidden_dims': [784, 512, 256, 128], 'apply_batchnorm': True, 'apply_activation': True, 'apply_dropout': True, 'num_classes': 10, 'dropout_ratio': 0.2}, 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.001, 'weight_decay': 0.0001}, 'scheduler': {'_target_': 'torch.optim.lr_scheduler.StepLR', 'step_size': 10, 'gamma': 0.5}, 'logger': {'project': 'lightning_with_hydra', 'name': 'test'}, 'callback': {'monitor': 'valid_loss', 'mode': 'min', 'patience': 3, 'logging_interval': 'epoch'}, 'trainer': {'max_epochs': 10, 'accelerator': 'auto'}}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9819999933242798
        test_loss          0.059492602944374084
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
최종 테스트 정확도: 0.9820
